In [ ]:
import requests
from pathlib import Path
# from matplotlib import pyplot as plt

## Credentials

In [ ]:
ID = 'ADD CLIENT_ID HERE'
TOKEN = 'ADD TOKEN HERE'

## Arguments

In [ ]:
BBOX = '16.180801,48.116142,16.578026,48.323387'  # http://bboxfinder.com/
OUT = 'output_images'
SIZE = 320  # {320, 640, 1024, 2048}

## Mounting Drive and Output Folder

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
OUT = '/content/gdrive/My Drive/WS_Materials/Street_View_Images'
Path(OUT).mkdir(parents=True, exist_ok=True)

In [ ]:
# First Request for Image Keys
url = f'https://a.mapillary.com/v3/images?bbox={BBOX}&client_id={ID}&per_page=1000'
headers = {'Authorization': f'Bearer {TOKEN}'}

# Counters
count, failed = 0, 0

# coords_x, coords_y = [], []

while url:
    response = requests.get(url, headers=headers)

    # Getting Next Page URL
    try:
        links = response.headers['link'].split(', ')
        for link in links:
            if 'rel="next"' in link:
                url = link[1:].split('>')[0]
            else:
                url = None
    except:
        url = None

    try:
        data = response.json()
        page_count = len(data["features"])
        count += page_count

        for i in range(page_count):

            if i % 10 == 0:
                # Getting Image Data
                image_data   = data['features'][i]
                image_key    = image_data['properties']['key']
                if image_data['properties']['ca']:
                    image_ca = str(int(image_data['properties']['ca']))
                image_coords = [str(x) for x in image_data['geometry']['coordinates']]
                image_url = f'https://images.mapillary.com/{image_key}/thumb-{SIZE}.jpg'

                # coords_x.append(image_data['geometry']['coordinates'][0])
                # coords_y.append(image_data['geometry']['coordinates'][1])

                # Downloading Images
                image_req = requests.get(image_url)
                if image_req.status_code == 200:
                    image_name = image_coords[0] + '_' + image_coords[1]
                    image_name += f'_{image_ca}' if image_ca else ''
                    with open(f'{OUT}/{image_name}.jpg', 'wb') as f:
                        f.write(image_req.content)
                        print(f'{i} | Downloaded image:{image_key} as {image_name}.jpg')
                else:
                    print(f'{i} | Ignored image:{image_key}')
                    failed += 1

    except Exception as e:
        print(e)

print(f'Done! Successfully downloaded {count - failed}/{count} images!')

# # Showing on plot
# plt.scatter(coords_x, coords_y)
# plt.show()

Developed by the City Intelligence Lab, Austrian Institute of Technology GmbH